Case:
> Basic

Feature Engineering
> None

Average Expenditures, Ammenity as high or low, Solo Travel

Read data, check dtypes, check for nulls

In [289]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as plt

df1 = pd.read_csv('train.csv')
y = df1['SalePrice'].values
df1 = df1.drop('SalePrice', axis=1)
df1['data'] = 1

In [290]:
df2 = pd.read_csv('test.csv')
df2['data'] = 2
y_id = df2['Id']
df = pd.concat([df1,df2], axis=0)
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,data
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,2
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,2
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,2
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,2


In [291]:
from prettytable import PrettyTable
synopsis = PrettyTable()
synopsis.field_names = ['Column', 'dtype', '%known', 'nan']
dtype = df.dtypes.to_list()
rows = len(df)
known = {}
nan = {}

i=0
for column in df.columns:
    nan[column] = df[column].isnull().sum()
    known[column] = rows-nan[column]
    synopsis.add_row([df.columns[i],dtype[i],known[column],nan[column]])
    i += 1

print(f"There are {rows} data points")
print(synopsis)

There are 2919 data points
+---------------+---------+--------+------+
|     Column    |  dtype  | %known | nan  |
+---------------+---------+--------+------+
|       Id      |  int64  |  2919  |  0   |
|   MSSubClass  |  int64  |  2919  |  0   |
|    MSZoning   |  object |  2915  |  4   |
|  LotFrontage  | float64 |  2433  | 486  |
|    LotArea    |  int64  |  2919  |  0   |
|     Street    |  object |  2919  |  0   |
|     Alley     |  object |  198   | 2721 |
|    LotShape   |  object |  2919  |  0   |
|  LandContour  |  object |  2919  |  0   |
|   Utilities   |  object |  2917  |  2   |
|   LotConfig   |  object |  2919  |  0   |
|   LandSlope   |  object |  2919  |  0   |
|  Neighborhood |  object |  2919  |  0   |
|   Condition1  |  object |  2919  |  0   |
|   Condition2  |  object |  2919  |  0   |
|    BldgType   |  object |  2919  |  0   |
|   HouseStyle  |  object |  2919  |  0   |
|  OverallQual  |  int64  |  2919  |  0   |
|  OverallCond  |  int64  |  2919  |  0   |
|   Y

### One-hot encoding

In [292]:
types = df.dtypes.to_list()
objects = []
numbers = []

for i in range(len(df.columns)):
    if types[i] == np.dtype('O'):
        objects.append(df.columns[i])
    else:
        numbers.append(df.columns[i])

unique = []
for i in range(len(objects)):
    items = df[objects[i]].unique().tolist()
    for j in range(len(items)):
        if pd.isna(items[j]):
            items.pop(j)
            break
    unique.append(items.copy())

for i in range(len(objects)):
    df[objects[i]] = df[objects[i]].replace(unique[i],list(range(1,len(unique[i])+1)))
    dist = df[objects[i]].value_counts(normalize=True)
    
    # case 1: impute by distribution
    # df.loc[df[objects[i]].isna(),objects[i]] = np.random.choice(dist.index, p=dist.values, size=nan[objects[i]])
    
    # case 2: impute to 0
    df[objects[i]] = df[objects[i]].replace([np.nan],[0])
    
    # case 3: impute by mode
    df[objects[i]] = df[objects[i]].replace([df[objects[i]].mode()],[0])


In [293]:
for i in range(len(numbers)):
    # case 1: mode
    # df[numbers[i]] = df[numbers[i]].replace([np.nan],[df[numbers[i]].mode()])
    # # case 2: mean
    df[numbers[i]] = df[numbers[i]].replace([np.nan],[df[numbers[i]].mean()])
    

In [294]:
df_drop = df.copy().dropna(axis=0)
rows2 = len(df_drop)
print(f'{(rows-rows2)/rows*100:0.2f}% of the data will be lost if dropped')


0.00% of the data will be lost if dropped


In [295]:
X = df.drop('Id', axis=1)
df1 = X[X['data']==1].drop('data',axis=1)
df2 = X[X['data']==2].drop('data',axis=1)
X1 = df1.values
X2 = df2.values

In [296]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

Model: LinearReg

In [297]:
lreg = LinearRegression()
lreg.fit(X1, y)


LinearRegression()

In [298]:
pred1 = lreg.predict(X2)
df1 = pd.DataFrame({'Id':y_id,'SalePrice':pred1}).to_csv('model1.csv',index=False)
# 0.22339
# 0.19691
# 0.34922
# 0.35024
# 0.21317
# 0.20957

Model: GradientBoost

In [299]:
from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor(random_state=1,n_estimators=250,learning_rate=0.1,max_depth=3)
gb.fit(X1,y)

GradientBoostingRegressor(n_estimators=250, random_state=1)

In [300]:
pred2 = gb.predict(X2)
df2 = pd.DataFrame({'Id':y_id,'SalePrice':pred2}).to_csv('model2.csv',index=False)
# 0.13241
# 0.13660
# 0.13453
# 0.13502
# 0.13489
# 0.13399

Model: CatBoost

In [301]:
from catboost import CatBoostRegressor
# cb=CatBoostRegressor(iterations=5000,verbose=0)
cb=CatBoostRegressor(random_state=1,n_estimators=250,learning_rate=0.1,max_depth=3,verbose=0)
cb.fit(X1,y)

pred3 = cb.predict(X2)
df3 = pd.DataFrame({'Id':y_id,'SalePrice':pred3}).to_csv('model3.csv',index=False)
# 0.13521
# 0.13540
# 0.13511
# 0.13526
# 0.13715

Model: XGBoost

In [302]:
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=X1,label=y)
xg= xgb.XGBRegressor(random_state=1,n_estimators=500,learning_rate=0.15,max_depth=5)

xg.fit(X1,y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.15, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=1, reg_alpha=0,
             reg_lambda=1, ...)

In [303]:
pred4 = xg.predict(X2)
df4 = pd.DataFrame({'Id':y_id,'SalePrice':pred4}).to_csv('model4.csv',index=False)
# 0.13928
# 0.13756
# 0.13830
# 0.13991
# 0.13568
# 0.13804

In [304]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

br = BaggingRegressor(DecisionTreeRegressor(max_depth=5), n_estimators=250)
br.fit(X1,y)

BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=5),
                 n_estimators=250)

In [305]:
pred5 = br.predict(X2)
df5 = pd.DataFrame({'Id':y_id,'SalePrice':pred5}).to_csv('model5.csv',index=False)
# 0.17871
# 0.17815
# 0.17952
# 0.18017
# 0.17760
# 0.17931


In [306]:
dtr = DecisionTreeRegressor(max_depth=5)
dtr.fit(X1,y)
pred6 = dtr.predict(X2)
df6 = pd.DataFrame({'Id':y_id,'SalePrice':pred6}).to_csv('model6.csv',index=False)
# 0.21467
# 0.21493
# 0.21329
# 0.21345
# 0.21513
# 0.21488

In [307]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(max_depth=5, 
                    random_state=1, 
                    n_estimators=250)
random_forest.fit(X1,y)

RandomForestRegressor(max_depth=5, n_estimators=250, random_state=1)

In [308]:
pred7 = random_forest.predict(X2)
df7 = pd.DataFrame({'Id':y_id,'SalePrice':pred7}).to_csv('model7.csv',index=False)

# 0.17812
# 0.17857
# 0.17828
# 0.17848
# 0.17818
# 0.17838
